## **News Research Tool**
 A Streamlit-based web application for equity research analysts to fetch and analyze news articles using AI.

In [17]:
!pip install langchain langchain-core langchain-groq feedparser python-dotenv streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 7.9 MB/s eta 0:00:00


In [ ]:
import os

os.environ["GROQ_API_KEY"] = your_groq_api_key_here


In [28]:
import feedparser
from urllib.parse import quote

def fetch_news(query, max_articles=5):
    encoded_query = quote(query)
    url = f"https://news.google.com/rss/search?q={encoded_query}&hl=en-IN&gl=IN&ceid=IN:en"

    feed = feedparser.parse(url)

    if not feed.entries:
        return "No relevant news found."

    text = ""
    for entry in feed.entries[:max_articles]:
        text += f"""
Title: {entry.title}
Summary: {entry.summary}
Source: {entry.get('source', {}).get('title', 'Google News')}
---
"""
    return text

In [34]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
import os

In [39]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3,
    groq_api_key=os.getenv("GROQ_API_KEY")
)

In [40]:
prompt = PromptTemplate(
    template="""
You are a senior equity research analyst.

User Query:
{query}

News Articles:
{news}

Tasks:
1. Give a short executive summary
2. Identify key trends
3. Highlight risks and opportunities
4. Keep output concise and factual
""",
    input_variables=["query", "news"]
)

chain = prompt | llm


In [41]:
def generate_summary(query):
    news_text = fetch_news(query)
    response = chain.invoke(
        {"query": query, "news": news_text}
    )
    return response.content

In [42]:
query = "Indian IT sector latest news"
result = generate_summary(query)
print(result)

**Executive Summary:**

The Indian IT sector is expected to continue its growth trajectory, with a report predicting it to cross $400 billion by 2030. However, the sector is also facing challenges, including Trump tariffs, which may impact exports. The sector's growth is expected to be driven by increasing demand for digital transformation and IT services.

**Key Trends:**

1. **Growing Demand for Digital Transformation:** The Indian IT sector is expected to benefit from increasing demand for digital transformation, driven by the need for businesses to adopt technology to remain competitive.
2. **Export Growth:** The sector's export growth is expected to be driven by increasing demand for IT services from developed markets.
3. **Tariff Risks:** The sector is facing risks due to Trump tariffs, which may impact exports.

**Risks and Opportunities:**

**Risks:**

1. **Tariff Risks:** Trump tariffs may impact exports, leading to reduced revenue and profitability.
2. **Competition:** The se

In [43]:
from ipywidgets import Text, Button, Output
from IPython.display import display

query_input = Text(
    description="Query:",
    placeholder="Example: Tesla earnings"
)

btn = Button(description="Analyze")
output = Output()

def on_click(b):
    with output:
        output.clear_output()
        print(generate_summary(query_input.value))

btn.on_click(on_click)

display(query_input, btn, output)

Text(value='', description='Query:', placeholder='Example: Tesla earnings')

Button(description='Analyze', style=ButtonStyle())

Output()